In [9]:
from pathlib import Path
import shutil
from tqdm import tqdm

#from kilosort.io import load_probe
from kilosort import run_kilosort, DEFAULT_SETTINGS
from kilosort.io import load_probe

SAVE_PATH = Path('/ix1/pmayo/Patient_FT_sEEG_Maria/FT_3_3_s.bin')
PROBE_PATH = '/ihome/pmayo/knoneman/Packages/HelperFunctions/mappings/probes/dixi1.json'

In [10]:
# quick probe maker
'''
import numpy as np

n_chan = 24
chanMap = np.arange(n_chan)
kcoords = np.zeros(n_chan)
xc = np.ones(n_chan)*7.5
yc = np.array([7.5 + 150*i for i in range(n_chan)])

probe = {
    'chanMap': chanMap,
    'xc': xc,
    'yc': yc[::-1],
    'kcoords': kcoords,
    'n_chan': n_chan
}

print(probe)
'''

"\nimport numpy as np\n\nn_chan = 24\nchanMap = np.arange(n_chan)\nkcoords = np.zeros(n_chan)\nxc = np.ones(n_chan)*7.5\nyc = np.array([7.5 + 150*i for i in range(n_chan)])\n\nprobe = {\n    'chanMap': chanMap,\n    'xc': xc,\n    'yc': yc[::-1],\n    'kcoords': kcoords,\n    'n_chan': n_chan\n}\n\nprint(probe)\n"

In [12]:
probe = load_probe(PROBE_PATH)

settings = DEFAULT_SETTINGS
print(settings)
settings['probe'] = probe
settings['n_chan_bin'] = 256
settings['fs'] = 30000
settings['data_dir'] = SAVE_PATH.parent

# spike detection
settings['batch_size'] = 30000 # 60000
settings['nblocks'] = 0 # 1
settings['Th_universal'] = 9 # 9
settings['Th_learned'] = 8   # 8

# clustering
settings['min_template_size'] = 10 #10
settings['nearest_templates'] = 100 # 100
#settings['max_channel_distance'] = 16 # None

kilosort_path = SAVE_PATH.parent / 'kilosort4'
temp_wh_path = SAVE_PATH.parent / 'temp_wh.dat'

if kilosort_path.exists() and kilosort_path.is_dir():
    shutil.rmtree(kilosort_path)

if temp_wh_path.exists() and temp_wh_path.is_file():
    temp_wh_path.unlink()    

{'n_chan_bin': 256, 'fs': 30000, 'batch_size': 60000, 'nblocks': 0, 'Th_universal': 9, 'Th_learned': 8, 'tmin': 0, 'tmax': inf, 'nt': 61, 'shift': None, 'scale': None, 'artifact_threshold': inf, 'nskip': 25, 'whitening_range': 32, 'highpass_cutoff': 300, 'binning_depth': 5, 'sig_interp': 20, 'drift_smoothing': [0.5, 0.5, 0.5], 'nt0min': None, 'dmin': None, 'dminx': 32, 'min_template_size': 10, 'template_sizes': 5, 'nearest_chans': 10, 'nearest_templates': 100, 'max_channel_distance': 32, 'max_peels': 100, 'templates_from_data': True, 'n_templates': 6, 'n_pcs': 6, 'Th_single_ch': 6, 'acg_threshold': 0.2, 'ccg_threshold': 0.25, 'cluster_downsampling': 20, 'x_centers': None, 'duplicate_spike_ms': 0.25, 'position_limit': 100, 'probe': {'chanMap': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,

In [13]:
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(settings=settings, probe=probe)

kilosort.run_kilosort: Kilosort version 0.1.dev1164+gdfaaf6b.d20240502
kilosort.run_kilosort: Python version 3.10.14
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: System information:
kilosort.run_kilosort: Linux-3.10.0-1160.71.1.el7.x86_64-x86_64-with-glibc2.17 x86_64
kilosort.run_kilosort: x86_64
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using CUDA device: NVIDIA A100-PCIE-40GB 39.38GB
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Sorting /ix1/pmayo/Patient_FT_sEEG_Maria/FT_3_3_s.bin
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 8869440
kilosort.run_kilosort: N seco

In [14]:
from kilosort.io import save_preprocessing, load_ops

# NOTE: This will only create the .dat file, it will *NOT* update options for Phy.
#       If you want to use this with Phy, you will need to modify `params.py`
#       in the results directory to point to this new file. Additionally,
#       you must set `hp_filtered=True` and `dtype='int16'`.
ops_path = SAVE_PATH.parent / 'kilosort4' / 'ops.npy'
ops = load_ops(ops_path)
save_preprocessing(SAVE_PATH.parent / 'temp_wh.dat', ops, bfile_path=SAVE_PATH)

kilosort.io :  
kilosort.io : ========================================
kilosort.io : Saving drift-corrected copy of data to: /ix1/pmayo/Patient_FT_sEEG_Maria/temp_wh.dat...
kilosort.io : Writing batch 0/296...
kilosort.io : Writing batch 100/296...
kilosort.io : Writing batch 200/296...
kilosort.io : ========================================
kilosort.io : Copying finished.
kilosort.io :  


In [15]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from kilosort.io import load_ops
from kilosort.data_tools import (
    mean_waveform, cluster_templates, get_good_cluster, get_cluster_spikes,
    get_spike_waveforms, get_best_channels
    )

# Indicate where sorting results were saved
results_dir = SAVE_PATH.parent / 'kilosort4'

# Pick a random good cluster
cluster_id = get_good_cluster(results_dir, n=1)

# Get the mean spike waveform and mean templates for the cluster
mean_wv, spike_subset = mean_waveform(cluster_id, results_dir, n_spikes=100,
                                      bfile=None, best=True)
scaled_temps = cluster_templates(cluster_id, results_dir, mean=False, best=True)
mean_temp = scaled_temps[spike_subset].mean(axis=0)

# Get time in ms for visualization
ops = load_ops(results_dir / 'ops.npy')
t = (np.arange(ops['nt']) / ops['fs']) * 1000

fig, ax = plt.subplots(1,1)
ax.plot(t, mean_wv, c='black', linestyle='dashed', linewidth=2, label='waveform')
ax.plot(t, mean_temp, linewidth=1, label='template')
ax.set_title(f'Mean single-channel template and spike waveform for cluster {cluster_id}')
ax.set_xlabel('Time (ms)')
ax.legend()

ValueError: too many values to unpack (expected 2)

In [8]:
%matplotlib ipympl

# Get n spike times for this cluster
spike_times, _ = get_cluster_spikes(cluster_id, results_dir, n_spikes=100)
# Time in s for spike time axis
t2 = spike_times / ops['fs']
# Get single-channel waveform for each spike
chan = get_best_channels(results_dir)[cluster_id]
waves = get_spike_waveforms(spike_times, results_dir, chan=chan)

# Plot each waveform, using spike time as 3rd dimension
fig, ax = plt.subplots(1, 1, figsize=(6,6), subplot_kw={'projection': '3d'})
for i in range(waves.shape[1]):
    # TODO: color by spike time
    ax.plot(t, t2[i], zs=waves[:,i], zdir='z');
ax.set_xlabel('Time (ms)');
ax.set_ylabel('Spike time (s)');
ax.view_init(azim=-100, elev=20);
ax.set_title(f'Spike waveforms for cluster {cluster_id}')
ax.set_box_aspect(None, zoom=0.85)

plt.tight_layout()

ValueError: too many values to unpack (expected 2)